In [ ]:
!pip3 install tinkoff-investments
!pip3 install yfinance
!pip3 install pandas
!pip3 install numpy
!pip3 install decimal
!pip3 install locale

In [39]:
from tinkoff.invest import Client

TOKEN = 't.b7eKSJEp3fpSiiv4mVt4fWwKIxaMHM1lDMtpGsPTeyl850b9Y4MluXYv-EQrj1vEu7QfkNwqGqGPfTW9N6EvTg'

import pandas as pd
from datetime import timedelta
from datetime import datetime, timedelta, timezone
from decimal import Decimal
from tinkoff.invest import CandleInterval, Client, HistoricCandle, Quotation, SubscriptionInterval
from tinkoff.invest.utils import now

__all__ = (
    "get_intervals",
    "quotation_to_decimal",
    "decimal_to_quotation",
    "candle_interval_to_subscription_interval",
    "now",
    "candle_interval_to_timedelta",
    "ceil_datetime",
    "floor_datetime",
    "dataclass_from_dict",
    "datetime_range_floor",
)


'''MoneyValue — используется для параметров, у которых есть денежный эквивалент. Возьмем для примера стоимость ценных бумаг — тип состоит из трех параметров:
1) currency — строковый ISO-код валюты, например RUB или USD;
2) units — целая часть суммы;
3) nano — дробная часть суммы, миллиардные доли единицы.
'''
# Quotation type = MoneyValue. We need to convert this to decimal in order to fetch price per share
def quotation_to_decimal(quotation: Quotation) -> Decimal:
    fractional = quotation.nano / Decimal("10e8")
    return Decimal(quotation.units) + fractional

def get_stock_volumes(_input: int):
    return f'{_input:,} ₽'

def get_final_float_stock_volumes(_input: int):
    return f'{_input:,} ₽'

def get_final_lots(_lots: int):
    return f'{_lots:,} шт.'

def calculate_net_change(current_closing_price: int, prev_closing_price: int):
    return f'Изменение цены: {round(((current_closing_price - prev_closing_price) / prev_closing_price * 100), 2)}%'

def calculate_net_change_per_day(current_closing_price: int, yesterday_closing_price: int):
    # current price minus 840 indexes in order to fetch price index yesterday for 1 minute candle
    return f'Изменение за день: {round(((current_closing_price - yesterday_closing_price) / yesterday_closing_price * 100), 2)}%'

def calculate_net_change_float(current_closing_price: float, prev_closing_price: float):
    return f'Изменение цены: {round(((current_closing_price - prev_closing_price) / prev_closing_price * 100), 2)}%'

def calculate_net_change_per_day_float(current_closing_price: float, yesterday_closing_price: float):
    # current price minus 840 indexes in order to fetch price index yesterday for 1 minute candle
    return f'Изменение за день: {round(((current_closing_price - yesterday_closing_price) / yesterday_closing_price * 100), 2)}%'

def make_million_volumes_on_float_stock_prices(price: int):
    price = str(price)
    price += '0000'
    return int(price)

def make_million_volumes_on_int_stock_prices(price: int):
    price = str(price)
    price += '0'
    return int(price)

def make_million_volumes_on_sngs(price: int):
    price = str(price)
    price += '000'
    return int(price)

def make_million_volumes_on_sngs(price: int):
    price = str(price)
    price += '00'
    return int(price)

l1_volumes = []
l2_lots = []
l3_prices = []
l4_time = []
l5_close = []
l6_high = []
l7_low = []
l8_bvp = []
l9_svp = []

def main():
    # vtbr BBG004730ZJ9
    # lkoh BBG004731032
    # yndx BBG006L8G4H1
    # mgnt BBG004RVFCY3
    # poly 	BBG004PYF2N3
    # sberp 	BBG0047315Y7
    # tcsg 	BBG00QPYJ5G1
    # chmf BBG00475K6C3
    # alrs BBG004S68B31
    # mmk 	BBG004S68507
    # phor 	BBG004S689R0
    # sngs 	BBG0047315D0
    # sngsp BBG004S681M2
    # nlmk 	BBG004S681B4
    with Client(TOKEN) as client:
        data = {"Объем": l1_volumes, "Лоты": l2_lots, "Цена": l3_prices, "Время": l4_time, "Закрытие": l5_close, "Хай": l6_high, "Лоу": l7_low, "Покупка": l8_bvp, "Продажа": l9_svp}
        threshold = 8.0
        for candle in client.get_all_candles(
            figi="BBG0047315D0",
            from_=now() - timedelta(days=90),
            interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
        ):
            if quotation_to_decimal(candle.close) < 1:
                # BUYING VOLUME AND SELLING VOLUME
                if candle.high == candle.low:
                    BV = 0
                    SV = 0
                else:
                    BV = (float(candle.volume) * (float(quotation_to_decimal(candle.close)) - float(quotation_to_decimal(candle.low)))) / (float(quotation_to_decimal(candle.high)) - float(quotation_to_decimal(candle.low)))
                    SV = (float(candle.volume) * (float(quotation_to_decimal(candle.high)) - float(quotation_to_decimal(candle.close)))) / (float(quotation_to_decimal(candle.high)) - float(quotation_to_decimal(candle.low)))
                    TP = BV + SV
                    BVP = round((BV / TP) * 100)
                    SVP = round((SV / TP) * 100)

                final_stock_volume_rub = int(candle.volume * float(quotation_to_decimal(candle.close)))
                l1_volumes.append(make_million_volumes_on_float_stock_prices(final_stock_volume_rub))
                l2_lots.append(get_final_lots(candle.volume)) # get_final_lots
                l3_prices.append(float(quotation_to_decimal(candle.close)))
                l4_time.append(candle.time)
                l5_close.append(float(quotation_to_decimal(candle.close)))
                l6_high.append(float(quotation_to_decimal(candle.high)))
                l7_low.append(float(quotation_to_decimal(candle.low)))
                l8_bvp.append(BVP)
                l9_svp.append(SVP)
                #print(f'ВТБ (VTBR), Время: {candle.time}, {get_final_lots(candle.volume)}, {get_final_float_stock_volumes(make_million_volumes_on_float_stock_prices(final_stock_volume_rub))}, Цена: {float(quotation_to_decimal(candle.close))} ₽')


            else:
                # BUYING VOLUME AND SELLING VOLUME
                if candle.high == candle.low:
                    BV = 0
                    SV = 0
                else:
                    BV = (float(candle.volume) * (float(quotation_to_decimal(candle.close)) - float(quotation_to_decimal(candle.low)))) / (float(quotation_to_decimal(candle.high)) - float(quotation_to_decimal(candle.low)))
                    SV = (float(candle.volume) * (float(quotation_to_decimal(candle.high)) - float(quotation_to_decimal(candle.close)))) / (float(quotation_to_decimal(candle.high)) - float(quotation_to_decimal(candle.low)))
                    TP = BV + SV
                    BVP = round((BV / TP) * 100)
                    SVP = round((SV / TP) * 100)
                #final_stock_volume_rub = int(candle.volume * quotation_to_decimal(candle.close))
                l1_volumes.append(int(candle.volume * quotation_to_decimal(candle.close)))
                l2_lots.append(get_final_lots(candle.volume)) # get_final_lots
                l3_prices.append(int(quotation_to_decimal(candle.close)))
                l4_time.append(candle.time)
                l5_close.append(float(quotation_to_decimal(candle.close)))
                l6_high.append(float(quotation_to_decimal(candle.high)))
                l7_low.append(float(quotation_to_decimal(candle.low)))
                l8_bvp.append(BVP)
                l9_svp.append(SVP)

                #print(f'Лукойл (LKOH), Время: {candle.time}, {get_final_lots(candle.volume)}, {get_final_stock_volumes(final_stock_volume_rub)}, Цена: {float(quotation_to_decimal(candle.close))} ₽')
        
        #df = pd.DataFrame(data)

        print(len(l1_volumes))
        print(len(l2_lots))
        print(len(l3_prices))
        print(len(l4_time))
        print(len(l5_close))
        print(len(l6_high))
        print(len(l7_low))
        print(len(l8_bvp))
        print(len(l9_svp))

        '''63324 VTBR
           63324
           63324
           63324
           63324
           63324
           63324
           63324
           63324'''
        
        '''53073'''

        # Calculate the rolling average and standard deviation of the trading volume
        '''volume_mean = round(df["Объем"].mean())
        volume_std = round(df["Объем"].std())

        # Calculate the rolling average and standard deviation of the trading prices
        prcies_mean = round(df["Цена"].mean())
        prices_std = round(df["Цена"].std())

        for i in range(len(df)):
            abnormal_volume = (df['Объем'].iloc[i] - volume_mean) / volume_std
            abnormal_price_changes = (df['Цена'].iloc[i] - prcies_mean) / prices_std
            #print(abnormal_volume, abnormal_price_changes)

            if abnormal_volume > threshold or abnormal_price_changes > threshold:
                if type(df["Цена"].iloc[i]) == float:
                    print(f'Зафиксирован аномальный объем! ВТБ (VTBR) {get_stock_volumes(df["Объем"].iloc[i])},  Цена: {df["Цена"].iloc[i]} ₽, {calculate_net_change_float(df["Цена"].iloc[i], df["Цена"].iloc[i - 1])}, {df["Время"].iloc[i]}, {calculate_net_change_per_day_float(df["Цена"].iloc[i], df["Цена"].iloc[i - 840])}, Время: {df["Время"].iloc[i]}')
                else:
                    if df["Хай"].iloc[i] == df["Лоу"].iloc[i]:
                        BV = 0
                        SV = 0
                    else:
                        BV = (df["Volumes"].iloc[i] * (df["Закрытие"].iloc[i] - df["Лоу"].iloc[i])) / (df["Хай"].iloc[i] - df["Закрытие"].iloc[i])
                        SV = (df["Volumes"].iloc[i] * (df["Хай"].iloc[i] - df["Закрытие"].iloc[i])) / (df["Хай"].iloc[i] - df["Лоу"].iloc[i])
                        TP = BV + SV
                        print(f'#YNDX Yandex\n🟩 Аномальный объем\n{calculate_net_change(df["Цена"].iloc[i], df["Цена"].iloc[i - 1])}\n{get_stock_volumes(df["Объем"].iloc[i])} {df["Лоты"].iloc[i]}\nПокупка: {round((BV / TP) * 100)}% Продажа: {round((SV / TP) * 100)}%\nВремя: {df["Время"].iloc[i]}\nЦена: {df["Цена"].iloc[i]} ₽\n{calculate_net_change_per_day(df["Цена"].iloc[i], df["Цена"].iloc[i - 840])}\nЗаметил Moex Oracle.')
                    #print(f'Зафиксирован аномальный объем! Лукойл (LKOH) {get_stock_volumes(df["Объем"].iloc[i])},  Цена: {df["Цена"].iloc[i]} ₽, {calculate_net_change(df["Цена"].iloc[i], df["Цена"].iloc[i - 1])}, {df["Время"].iloc[i]}, {calculate_net_change_per_day(df["Цена"].iloc[i], df["Цена"].iloc[i - 840])}, Время: {df["Время"].iloc[i]}')'''
        
    return 0

if __name__ == "__main__":
    main()


46884
46884
46884
46884
46884
46884
46884
46884
46884


In [40]:
data = {"Объем": l1_volumes, "Лоты": l2_lots, "Цена": l3_prices, "Время": l4_time, "Закрытие": l5_close, "Хай": l6_high, "Лоу": l7_low, "Покупка": l8_bvp, "Продажа": l9_svp}

df = pd.DataFrame(data)
THRESHOLD = 8.0
VOL_THRESHOLD = 30000000

# Calculate the rolling average and standard deviation of the trading volume
def calculate_volume_mean():
    volume_mean = df['Объем'].mean()
    return volume_mean

def calculate_volume_std():
    volume_std = df['Объем'].std()
    return volume_std

# Calculate the rolling average and standard deviation of the trading prices
def calculate_prices_mean():
    prcies_mean = df['Цена'].mean()
    return prcies_mean

def calculate_prices_std():
    prices_std = df['Цена'].std()
    return prices_std

#abnormal_volume = (df['Объем'].iloc[-1] - volume_mean) / volume_std

def calculate_net_change(current_closing_price: int, prev_closing_price: int):
    return f'Изменение цены: {round(((current_closing_price - prev_closing_price) / prev_closing_price * 100), 2)}%'

def calculate_net_change_per_day(current_closing_price: int, yesterday_closing_price: int):
    # current price minus 840 indexes in order to fetch price index yesterday for 1 minute candle
    return f'Изменение за день: {round(((current_closing_price - yesterday_closing_price) / yesterday_closing_price * 100), 2)}%'

def calculate_net_change_float(current_closing_price: float, prev_closing_price: float):
    return f'Изменение цены: {round(((current_closing_price - prev_closing_price) / prev_closing_price * 100), 2)}%'

def calculate_net_change_per_day_float(current_closing_price: float, yesterday_closing_price: float):
    # current price minus 840 indexes in order to fetch price index yesterday for 1 minute candle
    return f'Изменение за день: {round(((current_closing_price - yesterday_closing_price) / yesterday_closing_price * 100), 2)}%'

for i in range(len(df)):
    abnormal_volume = (df['Объем'].iloc[i] - calculate_volume_mean()) / calculate_volume_std()
    abnormal_price_changes = (df['Цена'].iloc[i] - calculate_prices_mean()) / calculate_prices_std()

    if abnormal_volume >= THRESHOLD or abnormal_price_changes > THRESHOLD:
        if df["Покупка"].iloc[i] > df["Продажа"].iloc[i]:
            print(f'#SNGS Сургутнефтегаз\n🟩 Аномальный объем\n{calculate_net_change(df["Цена"].iloc[i], df["Цена"].iloc[i - 1])}\n{get_stock_volumes(make_million_volumes_on_sngs(df["Объем"].iloc[i]))} ({df["Лоты"].iloc[i]})\nПокупка: {df["Покупка"].iloc[i]}% Продажа: {df["Продажа"].iloc[i]}%\nВремя: {df["Время"].iloc[i]}\nЦена: {df["Цена"].iloc[i]} ₽\n{calculate_net_change_per_day(df["Цена"].iloc[i], df["Цена"].iloc[i - 840])}\nЗаметил Moex Oracle.')
            print('=========================================')
        else:
            print(f'#SNGS Сургутнефтегаз\n🔻 Аномальный объем\n{calculate_net_change(df["Цена"].iloc[i], df["Цена"].iloc[i - 1])}\n{get_stock_volumes(make_million_volumes_on_sngs(df["Объем"].iloc[i]))} ({df["Лоты"].iloc[i]})\nПокупка: {df["Покупка"].iloc[i]}% Продажа: {df["Продажа"].iloc[i]}%\nВремя: {df["Время"].iloc[i]}\nЦена: {df["Цена"].iloc[i]} ₽\n{calculate_net_change_per_day(df["Цена"].iloc[i], df["Цена"].iloc[i - 840])}\nЗаметил Moex Oracle.')
            print('=========================================')

#SNGS Сургутнефтегаз
🟩 Аномальный объем
Изменение цены: 0.0%
38,026,900 ₽ (16,642 шт.)
Покупка: 92% Продажа: 8%
Время: 2023-02-28 11:50:00+00:00
Цена: 22 ₽
Изменение за день: 0.0%
Заметил Moex Oracle.
#SNGS Сургутнефтегаз
🔻 Аномальный объем
Изменение цены: 0.0%
24,899,000 ₽ (11,143 шт.)
Покупка: 31% Продажа: 69%
Время: 2023-03-02 10:15:00+00:00
Цена: 22 ₽
Изменение за день: 0.0%
Заметил Moex Oracle.
#SNGS Сургутнефтегаз
🟩 Аномальный объем
Изменение цены: 0.0%
25,691,600 ₽ (11,552 шт.)
Покупка: 53% Продажа: 47%
Время: 2023-03-16 11:58:00+00:00
Цена: 22 ₽
Изменение за день: 0.0%
Заметил Moex Oracle.
#SNGS Сургутнефтегаз
🟩 Аномальный объем
Изменение цены: 4.55%
27,581,600 ₽ (11,992 шт.)
Покупка: 100% Продажа: 0%
Время: 2023-03-20 08:32:00+00:00
Цена: 23 ₽
Изменение за день: 4.55%
Заметил Moex Oracle.
#SNGS Сургутнефтегаз
🟩 Аномальный объем
Изменение цены: 0.0%
33,154,500 ₽ (14,260 шт.)
Покупка: 94% Продажа: 6%
Время: 2023-03-20 15:26:00+00:00
Цена: 23 ₽
Изменение за день: 4.55%
Заметил Mo

In [ ]:
print(calculate_net_change(0.023075, 0.023055))
print(calculate_net_change_float(0.023075, 0.023055))

In [ ]:
import asyncio
import logging
import os
from datetime import timedelta
from typing import List, Optional

from tinkoff.invest import AioRequestError, AsyncClient, CandleInterval, HistoricCandle
from tinkoff.invest.async_services import AsyncServices
from tinkoff.invest.utils import now

TOKEN = 't.b7eKSJEp3fpSiiv4mVt4fWwKIxaMHM1lDMtpGsPTeyl850b9Y4MluXYv-EQrj1vEu7QfkNwqGqGPfTW9N6EvTg'


logging.basicConfig(format="%(asctime)s %(levelname)s:%(message)s", level=logging.DEBUG)
logger = logging.getLogger(__name__)


class LogOnlyCandlesStrategy:
    """This class is responsible for a strategy. You can put here
    your methods for your strategy."""

    def __init__(
        self,
        figi: str,
        timeframe: CandleInterval,
        days_back: int,
        check_interval: int,
        client: Optional[AsyncServices],
    ):
        self.account_id = None
        self.figi = figi
        self.timeframe = timeframe
        self.days_back = days_back
        self.check_interval = check_interval
        self.client = client
        self.candles: List[HistoricCandle] = []

    async def get_historical_data(self):
        """
        Gets historical data for the instrument. Returns list of candles.
        Requests all the candles of timeframe from days_back to now.

        :return: list of HistoricCandle
        """
        logger.debug(
            "Start getting historical data for %s days back from now. figi=%s",
            self.days_back,
            self.figi,
        )
        async for candle in self.client.get_all_candles(
            figi=self.figi,
            from_=now() - timedelta(days=self.days_back),
            to=now(),
            interval=self.timeframe,
        ):
            if candle not in self.candles:
                if candle.is_complete:
                    self.candles.append(candle)
                    logger.debug("Found %s - figi=%s", candle, self.figi)

    async def ensure_market_open(self):
        """
        Ensure that the market is open. Loop until the instrument is available.
        :return: when instrument is available for trading
        """
        trading_status = await self.client.market_data.get_trading_status(
            figi=self.figi
        )
        while not (
            trading_status.market_order_available_flag
            and trading_status.api_trade_available_flag
        ):
            logger.debug("Waiting for the market to open. figi=%s", self.figi)
            await asyncio.sleep(60)
            trading_status = await self.client.market_data.get_trading_status(
                figi=self.figi
            )

    async def main_cycle(self):
        """Main cycle for live strategy."""
        while True:
            try:
                await self.ensure_market_open()
                await self.get_historical_data()

                # put your strategy code here for live
                # to generate signals for buying or selling tickers
                logger.debug(
                    "- live mode: run some strategy code to buy or sell - figi=%s",
                    self.figi,
                )

            except AioRequestError as are:
                logger.error("Client error %s", are)

            await asyncio.sleep(self.check_interval)

    async def start(self):
        """Strategy starts from this function."""
        if self.account_id is None:
            try:
                self.account_id = (
                    (await self.client.users.get_accounts()).accounts.pop().id
                )
            except AioRequestError as are:
                logger.error("Error taking account id. Stopping strategy. %s", are)
                return
        await self.main_cycle()


async def run_strategy(portfolio, timeframe, days_back, check_interval):
    """From this function we are starting
    strategy for every ticker from portfolio.
    """
    async with AsyncClient(token=TOKEN, app_name="TinkoffApp") as client:
        strategy_tasks = []
        for instrument in portfolio:
            strategy = LogOnlyCandlesStrategy(
                figi=instrument,
                timeframe=timeframe,
                days_back=days_back,
                check_interval=check_interval,
                client=client,
            )
            strategy_tasks.append(asyncio.create_task(strategy.start()))
        await asyncio.gather(*strategy_tasks)


if __name__ == "__main__":

    vtbr_figi = "BBG004730ZJ9"
    portfolio = {vtbr_figi}
    timeframe = CandleInterval.CANDLE_INTERVAL_1_MIN
    days_back = 1
    check_interval = 10  # seconds to check interval for new completed candle

    loop = asyncio.get_event_loop()
    task = loop.create_task(
        run_strategy(
            portfolio=portfolio,
            timeframe=timeframe,
            days_back=days_back,
            check_interval=check_interval,
        )
    )
    loop.run_until_complete(task)


In [ ]:
vol = []
c = 0
while c <= 5:
    for i in range(100):
        vol.append(i)
        if len(vol) > 100:
            del vol[0]
    c += 1
print(len(vol))

In [ ]:
# 15,164,054
# 151,640,540

In [ ]:
import os
from datetime import timedelta
from decimal import Decimal
from tinkoff.invest import CandleInterval, Client, Quotation
from tinkoff.invest.utils import now

TOKEN = 't.b7eKSJEp3fpSiiv4mVt4fWwKIxaMHM1lDMtpGsPTeyl850b9Y4MluXYv-EQrj1vEu7QfkNwqGqGPfTW9N6EvTg'

# Quotation type = MoneyValue. We need to convert this to decimal in order to fetch price per share
def quotation_to_decimal(quotation: Quotation) -> Decimal:
    fractional = quotation.nano / Decimal("10e8")
    return Decimal(quotation.units) + fractional

def main():
    with Client(TOKEN) as client:
        for candle in client.get_all_candles(
            figi="BBG000R607Y3",
            from_=now() - timedelta(days=90),
            interval=CandleInterval.CANDLE_INTERVAL_1_MIN,
        ):
            print(f'Volume: {int(candle.volume * quotation_to_decimal(candle.close))}, {candle.time}, Price: {quotation_to_decimal(candle.close)}')

    return 0


if __name__ == "__main__":
    main()


In [ ]:
def make_million_volumes_on_int_stock_prices(price: int):
    price = str(price)
    price += '0'
    return int(price)

def make_million_volumes_on_sngs(price: int):
    price = str(price)
    price += '000'
    return int(price)

def get_stock_volumes(_input: int):
    return f'{_input:,} ₽'

vol_gazp: int = 85388
price_gazp: Decimal = 177.59000000

vol_lkoh: int = 17399
price_lkoh: Decimal = 5116.00000000

vol_sber: int = 63756054

print(f'GAZP {get_stock_volumes(make_million_volumes_on_int_stock_prices(round(vol_gazp * price_gazp)))}') # add make_million
print(f'LKOH {get_stock_volumes(round(vol_lkoh * price_lkoh))}') # not add make_million
print(f'SBER {get_stock_volumes(make_million_volumes_on_int_stock_prices(vol_sber))}') # add make_million
print(f'VTBR') # not add make_million
print(f'PLZL') # not add make_million
print(f'GMKN') # not add make_million
print(f'ROSN') # not add_make_million
print(f'NVTK') # not add_make_million
print(f'MGNT') # not add_make_million
print(f'POLY') # not add_make_million
print(f'SBERP') # add make_million
print(f'#CHMF (Северсталь)') # not add_make_million
print(f'#ALRS') # add make_million
print(f'#MAGN') # not add make_million
print(f'#PHOR') # not add make_million
print(f'#SNGS') # add make_million_sngs
print(f'#SNGSP') # add make_million_sngs
print(f'#NLMK') # # add make_million_on_int